# Experiment Introduction

Background, 
In order to do ablation experiment, we have diffculty like
1. most of them are same, we have to repeat many times.
2. too many model, config, code hard to manage and save. too messy


For all kind of SSL training workflow, we have to define the hyperparameters includes 4 aspect,
Dataset
Model
Training
Saving COnfig



How to Use this Experiment?



# Library

In [1]:
# Import config
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoSSL")
import yaml
from torchvision.transforms import RandomRotation,GaussianBlur,ColorJitter
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot,pipe_collate
from autoSSL.models import BarlowTwins, BYOL, MoCo, SimCLR, SimSiam, VICReg ,Toymodel, pipe_model 
from autoSSL.utils import embedding_feature,ck_callback,dict2transformer,join_dir,ContinuousCSVLogger  
from autoSSL.data import PipeDataset
from autoSSL.train import Trainer
 


C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import Global Baseline Config

In [2]:
# Load the YAML file
with open('global.yaml', 'r') as file:
    global_config = yaml.safe_load(file)

# Write your experiment notebook name here
global_config["experiment"]="paper_barlowtwins"   
global_config["experiment"]="paper_barlowtwins_xBD"   

global_config["prjhead_dim"]=[2048,2048]    
global_config["predhead_dim"]=[]
global_config["loss_func"]="BarlowTwinsLoss"     
global_config["view_model"]="None"     
global_config["view"]=2 
global_config["stop_gradient"]=False   
global_config["optimizer"]="LARS"      
global_config["schedule"]="cos"   
global_config["model"]="Toymodel"
global_config["batch_size"]=128
global_config["input_size"]=128
global_config["backbone"]="resnet18_pretrained"

# Define global view function
global_SSL_augmentation = {
    'RandomResizedCrop': {'size': global_config["input_size"], "scale":(0.2,1.0)},
    'RandomApply':{'transforms':[RandomRotation(degrees=90)], 'p':0},
    'RandomHorizontalFlip': {'p': 0.5},
    'RandomVerticalFlip':  {'p':0},
    'RandomApply':{'transforms': [ColorJitter(brightness=0.4,contrast=0.4,saturation=0.4,hue=0.1)], 'p':0.8},
    'RandomGrayscale' :{'p':0.2},
    #'RandomSolarize':{'threshold':128, 'p':0.1},
    'RandomApply':{'transforms':[GaussianBlur(kernel_size=3,sigma=(0.1, 2))],'p':0},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}

test_SSL_augmentation = {
    'RandomResizedCrop': {'size': global_config["input_size"], "scale":(0.2,1.0)},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
} 

# THIS IS THE CODE TO Monitor the KNN accuracy for each epoch
p_knndata= PipeDataset(input_dir=global_config["path_to_test_xBD"], 
        augmentation=dict2transformer(test_SSL_augmentation,view=1), 
        batch_size=global_config["batch_size"],num_workers=global_config["num_workers"]).dataloader
p_knndata=[p_knndata,5]  # The second number is the classes number of this datasets

# Grid Run Config

## Config1: Baseline of Cifar with Batch 512

In [ ]:

for sample in in [1]:
    
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"BT_res18_128_200"
    config["batch_size"]=128
    config["max_epochs"]=200
    config["input_size"]=512
    config["dataset_dir"]="../Datasets/xBD/train/"
    
    p_knndata= PipeDataset(input_dir=config["path_to_test_xBD"], 
        augmentation=dict2transformer(test_SSL_augmentation,view=1), 
        batch_size=config["batch_size"],num_workers=config["num_workers"]).dataloader
    p_knndata=[p_knndata,5]  # The second number is the classes number of this datasets
    
    
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5,log_every_n_steps=10) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    
    del pdata
    del pmodel
    del trainer
    

In [ ]:

for baseline in [0.5,0.25,0.1]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"BT_res50_64_100_{baseline}"
    config["batch_size"]=32
    config["max_epochs"]=100
    config["input_size"]=512
    config["dataset_dir"]="../Datasets/xBD/train/"
    config["samples"]= int(baseline*2799)
    global_config["backbone"]="resnet18_pretrained"
    p_knndata= PipeDataset(input_dir=config["path_to_test_xBD"], 
    augmentation=dict2transformer(test_SSL_augmentation,view=1), 
    batch_size=config["batch_size"],num_workers=config["num_workers"],samples=8*32).dataloader
    p_knndata=[p_knndata,5]  # The second number is the classes number of this datasets
    
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5,log_every_n_steps=int(2799*baseline/32/2)) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    
    del pdata
    del pmodel
    del trainer
     

## Config2: Without pretrained for 0.5

In [ ]:

for baseline in [["res18_random","resnet18"],["res50_random","resnet50"]]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"BT_{baseline[0]}_64_100_0.5"
    config["batch_size"]=32
    config["max_epochs"]=100
    config["input_size"]=512
    config["dataset_dir"]="../Datasets/xBD/train/"
    config["samples"]= int(0.5*2799)
    config["backbone"]=baseline[1]
    p_knndata= PipeDataset(input_dir=config["path_to_test_xBD"], 
    augmentation=dict2transformer(test_SSL_augmentation,view=1), 
    batch_size=config["batch_size"],num_workers=config["num_workers"],samples=8*32).dataloader
    p_knndata=[p_knndata,5]  # The second number is the classes number of this datasets
    
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5,log_every_n_steps=int(2799*0.5/32/2)) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    
    del pdata
    del pmodel
    del trainer
     

## Backbone RES 50: 100, 50, 25, 10

In [ ]:

for baseline in [0.1,0.25,0.5,1]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"BT_res50_64_100_{baseline}"
    config["batch_size"]=32
    config["max_epochs"]=100
    config["input_size"]=512
    config["dataset_dir"]="../Datasets/xBD/train/"
    config["samples"]= int(baseline*2799)
    global_config["backbone"]="resnet50_pretrained"
    p_knndata= PipeDataset(input_dir=config["path_to_test_xBD"], 
    augmentation=dict2transformer(test_SSL_augmentation,view=1), 
    batch_size=config["batch_size"],num_workers=config["num_workers"],samples=8*32).dataloader
    p_knndata=[p_knndata,5]  # The second number is the classes number of this datasets
    
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5,log_every_n_steps=int(2799*baseline/32/2)) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    
    del pdata
    del pmodel
    del trainer
     

# Evaluation

In [10]:
import os
import glob
import re
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
from autoSSL.models.Backbone import pipe_backbone
from sklearn.metrics import confusion_matrix
import seaborn as sns

def embed(x, embedding_model, device):
    embedding_model.eval()
    embedding_model.to(device)
    x = x.float().to(device)

    with torch.no_grad():
        embeddings = embedding_model(x)
        pooled_embeddings = torch.nn.functional.adaptive_avg_pool2d(embeddings, (1, 1))

    return pooled_embeddings.view(pooled_embeddings.size(0), -1).cpu().numpy()

def eval_linear(pipe_data, models, device='cuda', split=None, test=None, baseline=None):
    if split is not None:
        train_data, test_data = pipe_data.split(split)
    elif test is None:
        train_data = pipe_data
        test_data = pipe_data
    else:         
        train_data = pipe_data
        test_data = test

    print("Load the training and testing dataset")
    X_train, y_train = train_data.array[0], train_data.array[1]
    X_test, y_test = test_data.array[0], test_data.array[1]

    if isinstance(models, torch.nn.Module):
        models = {'name': ['model_0'], 'model': [models], 'address': None} 
    elif isinstance(models, list):
        models = {'name': ['model_'+str(i) for i in range(len(models))], 'model': models, 'address': None}
     
    baselines=[]
    results = []
    baselines_name=[]
    if baseline:
        for base in baseline:
            baseline_backbone, _ = pipe_backbone(backbone=base)
            models['model'].append(baseline_backbone)
            models['name'].append('baseline_' + base)
            baselines_name.append('baseline_' + base)
            baselines.append(baseline_backbone)
        
    writer = pd.ExcelWriter(models['address'].replace('.csv', '_confusion.xlsx'))

    for i, embedding_model in enumerate(tqdm(models['model'])):
        if embedding_model in baselines:
            pass
        else:
            embedding_model=embedding_model.backbone
        X_train_embedding = [embed(x, embedding_model, device) for x in DataLoader(X_train, batch_size=16)]
        X_train_embedding = np.concatenate(X_train_embedding)

        X_test_embedding = [embed(x, embedding_model, device) for x in DataLoader(X_test, batch_size=16)]
        X_test_embedding = np.concatenate(X_test_embedding)

        if X_test_embedding is None:
            accuracy = 'model_collapse'
            confusion = None
        else:
            clf = SGDClassifier(loss='log_loss')

            clf.fit(X_train_embedding, y_train)

            # Get class probabilities for each sample
            class_probs = clf.predict_proba(X_test_embedding)

            # Get the top 1 predictions
            top1_preds = np.argmax(class_probs, axis=1)
            top3_preds = np.argpartition(class_probs, -3, axis=1)[:,-3:]

            # Calculate confusion matrix
            confusion = confusion_matrix(y_test, top1_preds)
            # Normalize confusion matrix by row (i.e by the number of samples in each class)
            confusion = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis]
            # Average accuracy is the mean of the diagonal elements (the correctly classified instances)
            top1_average_accuracy = np.mean(np.diag(confusion))
            
            # Calculate accuracy
            top1_accuracy = accuracy_score(y_test, top1_preds)
            top3_accuracy = np.mean([1 if y in top3 else 0 for y, top3 in zip(y_test, top3_preds)])

            # K-Nearest Neighbors classifier
            knn = KNeighborsClassifier(n_neighbors=5)
            knn.fit(X_train_embedding, y_train)
            knn_preds = knn.predict(X_test_embedding)
            knn_accuracy = accuracy_score(y_test, knn_preds)

            accuracy = {
                "Top-1 Accuracy": top1_accuracy,
                "Top-3 Accuracy": top3_accuracy,
                "Top-1 Average Accuracy": top1_average_accuracy,
                "KNN Top-1 Accuracy": knn_accuracy  # KNN accuracy
            }

        namee = models["name"][i]
        results.append((namee, accuracy))

        
        del embedding_model
        torch.cuda.empty_cache()
        
        # Save confusion matrix to Excel file
        if confusion is not None:
            df_confusion = pd.DataFrame(confusion)
            df_confusion.to_excel(writer, sheet_name=namee)

    writer.save()

    if models['address'] is not None:
        df = pd.read_csv(models['address'])

        # If baselines are present, add new rows in the dataframe for them
        if baselines:
            for base, base_name in zip(baselines, baselines_name):
                # Initialize a new row with default values
                new_row = {col: None for col in df.columns}
                # Update the values we know
                new_row.update({
                    'dir_name': base_name,
                })
                # Append the new row to the dataframe
                df = df.append(new_row, ignore_index=True)

        # Assuming the results are in the same order as the models in the dataframe
        df['linear_top1_accuracy'] = [result[1]["Top-1 Accuracy"] for result in results]
        df['linear_top3_accuracy'] = [result[1]["Top-3 Accuracy"] for result in results]
        df['linear_top1_average_accuracy'] = [result[1]["Top-1 Average Accuracy"] for result in results]
        df['linear_knn_top1_accuracy'] = [result[1]["KNN Top-1 Accuracy"] for result in results]

        df.to_csv(models['address'], index=False)

    return results


In [11]:

test_augmentation = {
    'RandomResizedCrop': {'size': (global_config["input_size"], global_config["input_size"])},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}
collate =pipe_collate(address="experiment_checkpoints/paper_barlowtwins_xBD/", reg=f"^BT_res50_64_100_.+$")
test_augmentation = {
    #'RandomResizedCrop': {'size': global_config["input_size"], "scale":(0.2,1.0)},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
} 

pdata = PipeDataset(input_dir=global_config["path_to_test_xBD"],   #
    augmentation=dict2transformer(test_augmentation,view=1)) #,samples=100


Collating the models' (evaluating) information to experiment_checkpoints/paper_barlowtwins_xBD/^BT_res50_64_100_.+$.csv


In [12]:
aaa=eval_linear(pdata, models=collate, device=global_config["device"],baseline=["resnet18","resnet18_pretrained"],split=0.7)


Load the training and testing dataset


 17%|██████████████                                                                      | 1/6 [00:05<00:27,  5.49s/it]C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\sklearn\linear_model\_base.py:440: RuntimeWarning: invalid value encountered in divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))
 33%|████████████████████████████                                                        | 2/6 [00:16<00:35,  8.90s/it]C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\sklearn\linear_model\_base.py:440: RuntimeWarning: invalid value encountered in divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))
 50%|██████████████████████████████████████████                                          | 3/6 [00:28<00:30, 10.06s/it]C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\sklearn\linear_model\_base.py:440: RuntimeWarning: invalid value encountered in divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))
100%|█████████████████████████████████████████

In [13]:
aaa

[('BT_res50_64_100_0.1',
  {'Top-1 Accuracy': 0.32857142857142857,
   'Top-3 Accuracy': 0.8285714285714286,
   'Top-1 Average Accuracy': 0.35995708947199306,
   'KNN Top-1 Accuracy': 0.5}),
 ('BT_res50_64_100_0.25',
  {'Top-1 Accuracy': 0.475,
   'Top-3 Accuracy': 0.8714285714285714,
   'Top-1 Average Accuracy': 0.46320776437667144,
   'KNN Top-1 Accuracy': 0.5428571428571428}),
 ('BT_res50_64_100_0.5',
  {'Top-1 Accuracy': 0.525,
   'Top-3 Accuracy': 0.8785714285714286,
   'Top-1 Average Accuracy': 0.49519913166903234,
   'KNN Top-1 Accuracy': 0.5571428571428572}),
 ('BT_res50_64_100_1',
  {'Top-1 Accuracy': 0.5107142857142857,
   'Top-3 Accuracy': 0.8642857142857143,
   'Top-1 Average Accuracy': 0.5047861937400219,
   'KNN Top-1 Accuracy': 0.5678571428571428}),
 ('baseline_resnet18',
  {'Top-1 Accuracy': 0.3821428571428571,
   'Top-3 Accuracy': 0.6678571428571428,
   'Top-1 Average Accuracy': 0.4069765029437735,
   'KNN Top-1 Accuracy': 0.42142857142857143}),
 ('baseline_resnet18_pre

In [ ]:
 
test_augmentation = {
    'RandomResizedCrop': {'size': (global_config["input_size"], global_config["input_size"])},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}
collate =pipe_collate(address="experiment_checkpoints/paper_barlowtwins_xBD/", reg="BT_res50_64_100_[0-9]+")

pdata = PipeDataset(input_dir=global_config["path_to_test_xBD"],
    augmentation=dict2transformer(test_augmentation,view=1))


In [ ]:
aaa=eval_linear(pdata, models=collate, device=global_config["device"])
